In [1]:
#read datafolder from Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/My Drive/pg2k18/sem2/smai/smai_proj/reuters21578/'


sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)


In [4]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
lemmatizer = WordNetLemmatizer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def cleanUpSentence(r):#, stop_words = None#
    r = r.lower().replace("<br />", " ")
    r = REPLACE_BY_SPACE_RE.sub(' ', r)
    r = BAD_SYMBOLS_RE.sub('', r)

    r = ' '.join(word for word in r.split() if word not in STOPWORDS)

    words = word_tokenize(r)

    for w in words:
        w = lemmatizer.lemmatize(w)

    return r

In [0]:
# Parse SGML files
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
  
def makeDict(filename, document_X, document_Y):
  with open(filename, 'rb') as file:

    content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
      document_categories = []

      document_id = newsline['newid']



      document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
      if document_body == 'None':
        continue

      doc_categories = strip_tags(str(newsline('topics')[0].body))
      doc_categories = unescape(doc_categories)

      document_body = unescape(document_body)

      topics = newsline.topics.contents
      places = newsline.places.contents
      people = newsline.people.contents
      orgs = newsline.orgs.contents
      exchanges = newsline.exchanges.contents

      for topic in topics:
          document_categories.append('to_' + strip_tags(str(topic)))

      for place in places:
          document_categories.append('pl_' + strip_tags(str(place)))

      for person in people:
          document_categories.append('pe_' + strip_tags(str(person)))

      for org in orgs:
          document_categories.append('or_' + strip_tags(str(org)))

      for exchange in exchanges:
          document_categories.append('ex_' + strip_tags(str(exchange)))

      document_X[document_id] = document_body
      document_Y[document_id] = document_categories

def readFiles(test_data = False):
  document_X = {}
  document_Y = {}
  if test_data == True:
    file_name = sgml_file_name_template.replace('NNN', '021')
    filename = data_folder + file_name
    makeDict(filename, document_X, document_Y)
  else:
    for i in range(sgml_number_of_files):
      if i < 10:
        seq = '00' + str(i)
      else:
        seq = '0' + str(i)

      file_name = sgml_file_name_template.replace('NNN', seq)
      print('Reading file: %s' % file_name)
      filename = data_folder + file_name
      makeDict(filename, document_X, document_Y)
  return document_X, document_Y



In [7]:
document_X, document_Y = readFiles()

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm


In [8]:
# data preprocessing
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
totalX,encoded_docs=create_x_matrix(document_X)

In [0]:
#Creating one-hot encode
words_in_body={}

for i in range(len(totalX)):
    words=totalX[i].split(' ')
    words_in_body[i]=words    

one_hot_label=[]
for key,v in words_in_body.items():
    dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
    for i in v:
        if i in category_dictionary['Topics']:
            dict_temp['Topics']+=1
        if i in category_dictionary['Places']:
            dict_temp['Places']+=1
        if i in category_dictionary['People']:
            dict_temp['People']+=1
        if i in category_dictionary['Exchanges']:
            dict_temp['Exchanges']+=1
        if i in category_dictionary['Organizations']:
            dict_temp['Organizations']+=1
            
    one_hot_label.append(dict_temp)
    

one_hot_label_list = []
for i in one_hot_label:

    one_hot_label_list.append(list(i.values()))



In [16]:
from keras.models import Sequential
from keras.layers import Dense,Flatten, Dropout,Embedding
nn = Sequential()
max_vocab_size = 200
nn.add(Embedding(200, 20, input_length=max_vocab_size))
nn.add(Dense(10, activation="relu", input_shape=(max_vocab_size,)))
nn.add(Dropout(0.15))
nn.add(Dense(15, activation="relu"))
nn.add(Dropout(0.15))
nn.add(Flatten())
nn.add(Dense(5,activation="softmax"))
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(np.array(encoded_docs), np.array(one_hot_label_list), batch_size=16, epochs=5,
          verbose=1, validation_split=0.2)

Train on 14866 samples, validate on 3717 samples
Epoch 1/5
14866/14866 [==============================] - 5s 345us/step - loss: 3.1435 - acc: 0.8011 - val_loss: 3.1818 - val_acc: 0.7980
Epoch 2/5
14866/14866 [==============================] - 5s 332us/step - loss: 2.8266 - acc: 0.8092 - val_loss: 3.1264 - val_acc: 0.7958
Epoch 3/5
14866/14866 [==============================] - 5s 331us/step - loss: 2.7742 - acc: 0.8138 - val_loss: 3.1058 - val_acc: 0.7807
Epoch 4/5
14866/14866 [==============================] - 5s 331us/step - loss: 2.7487 - acc: 0.8137 - val_loss: 3.0826 - val_acc: 0.8023
Epoch 5/5
14866/14866 [==============================] - 5s 331us/step - loss: 2.7242 - acc: 0.8137 - val_loss: 3.0836 - val_acc: 0.7931


In [0]:
test_document_X, test_document_Y = readFiles(test_data=True)
# print(len(test_document_X))

In [0]:
test_total_X,test_encoded_X= create_x_matrix(test_document_X)
y = nn.predict(test_encoded_X)
# print (y)

In [15]:
def getRankedOutput2(predicted_y):
    all_sorted_x = []
    for i in predicted_y:
        dict_temp={'Topics':i[0],'Places':i[1],'Peoples':i[2],'Exchanges':i[3],'Organizations':i[4]}
        sorted_x = sorted(dict_temp.items(), key=lambda kv: kv[1], reverse=True) 
        all_sorted_x.append(sorted_x)
    return all_sorted_x
   
ranked_output = getRankedOutput2(y)
print((ranked_output)) 

[[('Topics', 0.53592527), ('Places', 0.4392004), ('Peoples', 0.02442408), ('Exchanges', 0.00032594224), ('Organizations', 0.00012428484)], [('Places', 0.6268859), ('Topics', 0.22719973), ('Organizations', 0.079037115), ('Peoples', 0.052243378), ('Exchanges', 0.0146338325)], [('Topics', 0.66584396), ('Exchanges', 0.15620339), ('Places', 0.12874521), ('Peoples', 0.033515878), ('Organizations', 0.015691597)], [('Topics', 0.5822374), ('Places', 0.2442483), ('Peoples', 0.16485062), ('Organizations', 0.0054216124), ('Exchanges', 0.00324199)], [('Topics', 0.5361135), ('Exchanges', 0.29234073), ('Places', 0.12338199), ('Peoples', 0.03325075), ('Organizations', 0.01491299)], [('Topics', 0.6437268), ('Places', 0.24993171), ('Peoples', 0.07437792), ('Organizations', 0.01634745), ('Exchanges', 0.015616068)], [('Topics', 0.84294426), ('Places', 0.12105716), ('Organizations', 0.016590066), ('Peoples', 0.0111146355), ('Exchanges', 0.008293882)], [('Topics', 0.8304456), ('Places', 0.0906116), ('People